In [8]:
import torch
from torch import nn

## Basic Block
* 18, 34 layer

In [9]:
class BasicBlock(nn.Module):

    # 나가는 채널을 몇 배 늘릴지 결정
    expansion = 1

    def __init__(self, in_channels, inner_channels, stride=1, projection=None):
        super().__init__()

        # 3x3 conv 두 번 통과 => F(x) 역할
        self.residual = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=inner_channels,
                kernel_size=3,
                stride=stride,
                padding=1,
                bias=False,
            ),
            nn.BatchNorm2d(num_features=inner_channels),
            nn.ReLU(inplace=True),  # inplace 설정 시 메모리 상의 이득
            nn.Conv2d(
                in_channels=inner_channels,
                out_channels=inner_channels * self.expansion,
                kernel_size=3,
                padding=1,
                bias=False,
            ),
            nn.BatchNorm2d(inner_channels * self.expansion),
        )

        # projection 은 1x1 conv 진행
        self.projection = projection
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        # F(x)부터 계산
        residual = self.residual(x)

        # skip connection
        if self.projection is not None:
            # 이전 스테이지의 마지막 블록 출력의 채널 두배, 가로세로 절반
            shortcut = self.projection(x)
        else:
            shortcut = x

        out = self.relu(residual + shortcut)

        return out

## BottleNeck 
* 50, 101, 152 layer

In [10]:
class BottleNeck(nn.Module):
    # 내보낼 때 채널이 4배로 증가
    expansion = 4

    def __init__(self, in_channels, inner_channels, stride=1, projection=None):
        super().__init__()

        # 1x1 -> 3x3 -> 1x1
        self.residual = nn.Sequential(
            # 1x1
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=inner_channels,
                kernel_size=1,
                bias=False,
            ),
            nn.BatchNorm2d(num_features=inner_channels),
            nn.ReLU(inplace=True),
            # 3x3
            nn.Conv2d(
                in_channels=inner_channels,
                out_channels=inner_channels,
                kernel_size=3,
                stride=stride,
                padding=1,
                bias=False,
            ),
            nn.BatchNorm2d(num_features=inner_channels),
            nn.ReLU(inplace=True),
            # 1x1
            nn.Conv2d(
                in_channels=inner_channels,
                out_channels=inner_channels * self.expansion,
                kernel_size=1,
                bias=False,
            ),
            nn.BatchNorm2d(num_features=inner_channels * self.expansion),
        )
        self.projection = projection
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = self.residual(x)

        if self.projection is not None:
            shortcut = self.projection(x)
        else:
            shortcut = x

        out = self.relu(residual + shortcut)

        return out

## ResNet module

In [11]:
class ResNet(nn.Module):
    def __init__(
        self, block, num_block_list, num_classes=1000, zero_init_residual=True
    ):
        super().__init__()

        self.in_channels = 64

        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=64,
            kernel_size=7,
            stride=2,
            padding=3,
            bias=False,
        )
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.stage1 = self.make_stage(block, 64, num_block_list[0], stride=1)
        self.stage2 = self.make_stage(block, 128, num_block_list[1], stride=2)
        self.stage3 = self.make_stage(block, 256, num_block_list[2], stride=2)
        self.stage4 = self.make_stage(block, 512, num_block_list[3], stride=2)

        if zero_init_residual:
            # 전체 모듈 가져오기
            for m in self.modules():
                # if module is a block (BottleNeck or BasicBlock)
                if isinstance(m, block):
                    # residual 모듈의 마지막 레이어의 가중치를 0으로
                    # residual 모듈의 마지막 레이어에 batch normalization
                    # 각 모듈의 마지막 층은 BatchNorm2d입니다. 이 BatchNorm2d의 weight 파라미터(γ 또는 alpha라고도 부릅니다)를 0으로 초기화합니다.
                    # γ를 0으로 설정하면 해당 BatchNorm 층의 출력이 0이 되어 residual branch의 전체 출력이 0이 됩니다.
                    # 따라서 이 residual branch는 초기에는 출력을 하지 않고, 블록 전체는 입력을 그대로 출력하는 identity mapping처럼 동작합니다.
                    # 이는 초기 학습 시 네트워크의 안정적인 수렴을 도와주며, 모델의 성능을 향상시킵니다.
                    # 이러한 방법은 논문 https://arxiv.org/abs/1706.02677 에서 제안되었으며, 약 0.2~0.3%의 성능 향상이 있다고 합니다.
                    nn.init.constant_(m.residual[-1].weight, 0)  # BN 의 가중치를 0으로

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def forward(self, x):
        # 입력된 이미지에 대한 기본 처리
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        # 잔차 학습 (스테이지 통과)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)

        # FCL
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        y = self.fc(x)

        return y

    def make_stage(self, block, inner_channels, num_blocks, stride=1):
        if stride != 1 or self.in_channels != inner_channels * block.expansion:
            projection = nn.Sequential(
                nn.Conv2d(
                    in_channels=self.in_channels,
                    out_channels=inner_channels * block.expansion,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                ),
                nn.BatchNorm2d(num_features=inner_channels * block.expansion),
            )
        else:
            projection = None

        layers = []

        # 각 stage 의 첫 layer 는 projection 수행
        layers += [block(self.in_channels, inner_channels, stride, projection)]
        self.in_channels = inner_channels * block.expansion

        for _ in range(1, num_blocks):
            layers += [
                block(self.in_channels, inner_channels)
            ]  # 첫 layer 이후는 stride, projection 설정 x

        # * : iterable 객체를 unpacking
        # ** : dictionary 객체를 unpacking
        return nn.Sequential(*layers)

In [12]:
def resnet18(**kwargs):
    return ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)


def resnet34(**kwargs):
    return ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)


def resnet50(**kwargs):
    return ResNet(BottleNeck, [3, 4, 6, 3], **kwargs)


def resnet101(**kwargs):
    return ResNet(BottleNeck, [3, 4, 23, 3], **kwargs)


def resnet152(**kwargs):
    return ResNet(BottleNeck, [3, 8, 36, 3], **kwargs)

In [13]:
model = resnet50()

In [14]:
from torchinfo import summary

summary(model, input_size=(64, 3, 224, 224), device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [64, 1000]                --
├─Conv2d: 1-1                            [64, 64, 112, 112]        9,408
├─BatchNorm2d: 1-2                       [64, 64, 112, 112]        128
├─ReLU: 1-3                              [64, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [64, 64, 56, 56]          --
├─Sequential: 1-5                        [64, 256, 56, 56]         --
│    └─BottleNeck: 2-1                   [64, 256, 56, 56]         --
│    │    └─Sequential: 3-1              [64, 256, 56, 56]         58,112
│    │    └─Sequential: 3-2              [64, 256, 56, 56]         16,896
│    │    └─ReLU: 3-3                    [64, 256, 56, 56]         --
│    └─BottleNeck: 2-2                   [64, 256, 56, 56]         --
│    │    └─Sequential: 3-4              [64, 256, 56, 56]         70,400
│    │    └─ReLU: 3-5                    [64, 256, 56, 56]         --